In [1]:
from model import Chatbot, Encoder, Decoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from pathlib import Path
from unidecode import unidecode
import numpy as np
import pickle

In [2]:
data_dir = Path('data')

with open(data_dir/'preguntas.txt', 'r', encoding='utf-8') as f:
    questions = f.readlines()

with open(data_dir/'respuestas.txt', 'r', encoding='utf-8') as f:
    answers = f.readlines()

print(f"El archivo 'preguntas.txt' contiene {len(questions)} preguntas")

El archivo 'preguntas.txt' contiene 273 preguntas


In [3]:
START_TOKEN = '<START> '
END_TOKEN = ' <END>'

processed_questions = list(map(lambda x: unidecode(x.lower()), questions))
processed_answers = list(map(lambda x: START_TOKEN + unidecode(x.lower().strip()) + END_TOKEN, answers))

print(processed_answers[0])

<START> "rosebud" <END>


In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(processed_questions + processed_answers)
vocab_size = len(tokenizer.word_index)

print(f"There are {vocab_size} unique words")

There are 1725 unique words


In [5]:
# Preparing Encoder Input
tokenized_questions = tokenizer.texts_to_sequences(processed_questions)
max_input_length = max(list(map(len, tokenizer.texts_to_sequences(processed_questions))))
encoder_input = np.array(pad_sequences(tokenized_questions, maxlen=max_input_length, padding='post'))
print(f"Encoder input data shape: {encoder_input.shape}")

# Preparing Decoder Input
tokenized_answers = tokenizer.texts_to_sequences(processed_answers)
max_output_length = max(list(map(len, tokenizer.texts_to_sequences(processed_answers))))
decoder_input = np.array(pad_sequences(tokenized_answers, maxlen=max_input_length, padding='post'))
print(f"Decoder input data shape: {decoder_input.shape}")

# Preparing Target Output
tokenized_output = list(map(lambda x: x[1:], tokenized_answers))
padded_target = pad_sequences(tokenized_output, maxlen=max_input_length, padding='post')
decoder_target = np.array(to_categorical(padded_target))
print(f"Decoder target data shape: {decoder_target.shape}")

with open(data_dir/'tokenizer.pkl', 'wb') as f:
    pickle.dump({
        'tokenizer': tokenizer,
        'vocab_size': vocab_size,
        'max_input_length': max_input_length,
        'max_output_length': max_output_length,
        'model_bootstrap': (encoder_input[:1], decoder_input[:1], decoder_target[:1])   
    }, f, protocol=pickle.DEFAULT_PROTOCOL)

Encoder input data shape: (273, 31)
Decoder input data shape: (273, 31)
Decoder target data shape: (273, 31, 1726)


In [6]:
model = Chatbot(vocab_size + 1)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input, decoder_input], decoder_target, batch_size=10, epochs=100)

Epoch 1/100
28/28 [==============================] - 1s 19ms/step - loss: 1.7571 - accuracy: 0.1644
Epoch 2/100
28/28 [==============================] - 1s 19ms/step - loss: 1.4012 - accuracy: 0.2011 0s - loss: 1.4524 - 
Epoch 3/100
28/28 [==============================] - 1s 20ms/step - loss: 1.2828 - accuracy: 0.2337
Epoch 4/100
28/28 [==============================] - 1s 20ms/step - loss: 1.2369 - accuracy: 0.2529
Epoch 5/100
28/28 [==============================] - 1s 21ms/step - loss: 1.1930 - accuracy: 0.2650
Epoch 6/100
28/28 [==============================] - 1s 20ms/step - loss: 1.1320 - accuracy: 0.2708
Epoch 7/100
28/28 [==============================] - 1s 19ms/step - loss: 1.0851 - accuracy: 0.2873
Epoch 8/100
28/28 [==============================] - 1s 20ms/step - loss: 1.0424 - accuracy: 0.3003
Epoch 9/100
28/28 [==============================] - 1s 20ms/step - loss: 1.0007 - accuracy: 0.3320
Epoch 10/100
28/28 [==============================] - 1s 19ms/step - loss: 0.95

In [7]:
model.save_weights(Path('model')/'model.hdf5')

In [8]:
def preprocess_input(text: str):
    text = text.lower()
    text = unidecode(text)
    tokenized_text = tokenizer.texts_to_sequences([text])
    padded_text = pad_sequences(tokenized_text, maxlen=max_input_length, padding='post')

    return padded_text

In [9]:
#model = Chatbot(vocab_size + 1)
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model.train_on_batch([encoder_input[:1], decoder_input[:1]], decoder_target[:1])
#model.load_weights(Path('model')/'model.hdf5')

In [19]:
question = 'Qué pelicula de Alfred Hitchcock se rodo como si fuera una toma continua'
empty_target_seq = np.array([[tokenizer.word_index['start']]])
stop_condition = False
answer = ''
stop_words = ['adios', 'gracias']

proc_question = preprocess_input(question)
encoder_input = model.embedding(proc_question)
h, c = model.encoder.predict([encoder_input])

while not stop_condition:
    decoder_input = model.embedding(empty_target_seq)
    decoder_output, h, c = model.decoder.predict([decoder_input, h, c])
    sampled_word_index = np.argmax( decoder_output[0, -1, :] )
    sampled_word = tokenizer.index_word.get(sampled_word_index, None)
    if sampled_word == 'end' or len(answer.split()) > max_output_length:
        stop_condition = True
    else:
        answer += f"{sampled_word} "
    

    empty_target_seq = np.array([np.append(empty_target_seq[0],sampled_word_index)])
    enc_stats = [h, c]


print(answer)


la soga 
